In [ ]:
import SimpleITK as sitk
from pathlib import Path

In [ ]:
# Directory that holds the fixed image.  Copy paste the windows file path inbetween the parenthesis.  E.g., r'C:\users\alex'
fixed_dir = Path(r'')
fixed_path = Path(fixed_dir, 'Fixed image name.tif')

moving_dir = Path(r'')
moving_path = Path(moving_dir, 'Moving image name.tif')

output_dir = Path(r'')
output_path = Path(output_dir, 'Output image name.tif')

In [ ]:
# You need to manually set the image pixel spacing in microns to get a good registration.  2 numbers for 2D images, 3 for 3D
fixed_spacing = [1, 1]
moving_spacing = [1, 1]

In [ ]:
# This gives you a rigid 2D transform
initial_transform = sitk.Euler2DTransform()

# if you need to do 3D, use this for rigid 3d.
# initial_transform = sitk.VersorRigid3DTransform()

In [ ]:
# Read in the fixed image
fixed_image = sitk.ReadImage(str(fixed_path))
fixed_image.SetSpacing(fixed_spacing)

In [ ]:
# Read in the moving image
moving_image = sitk.ReadImage(str(moving_path))
moving_image.SetSpacing(moving_spacing)

In [ ]:
# Define how the registration works in the background.  
registration_method = sitk.ImageRegistrationMethod()

registration_method.SetMetricAsMattesMutualInformation()
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(1)

registration_method.SetInterpolator(sitk.sitkLinear)
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, 
                                                  convergenceMinimumValue=1e-6, convergenceWindowSize=10)

In [ ]:
# Run the registration to calculate a transform
final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32),
                                              sitk.Cast(moving_image, sitk.sitkFloat32))

In [ ]:
# Resample the moving image to the registered result
registered_image = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

In [ ]:
#Save the final image.
sitk.WriteImage(registered_image, str(output_path))